# Iteration 4: Check the sampling from lognormal distributions stage 2

## Plain English Summary

In this iteration we fix the sampling from lognormal distributions.  We create a function that converts the mean and standard deviation of the lognormal to the equivalent parameters of the underlying normal distribution.  This is because `numpy` requires the parameters to be in this format.

## Notebook imports

In [1]:
from IPython.display import Markdown, display
import math
from sim_tools.distributions import Lognormal

## LLM Prompt

In [2]:
display(Markdown("../01_perplexity/inputs/04_iteration.md"))

### Main command

modify all `acute_treatment `functions to call a function external to the class called `normal_moments_from_lognormal`. The function should accept the mean and standard deviation of a Lognormal distribution and return the mu, sigma of the underlying Normal distribution. The returned values are the correct values to use when sampling from the Lognormal distributions. 


Show all of the model code: this includes all patient generator functions in the ASU class.

## Date and time of input to LLM

> 1315 28th June 2024

## Observations on the LLM Output

* `normal_moments_from_lognormal` is defined correctly. 

## LLM Output 
> Carried over from previous iteration, as a function was created then without explicitly being asked.
> Here it is being tested before using the prompt for iteration 4.

In [3]:
import simpy
import numpy as np
import math

class PatientType:
    def __init__(self, name, interarrival_time, post_asu_probabilities, los_params):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()
        self.post_asu_probabilities = post_asu_probabilities
        self.los_params = los_params

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

    def sample_post_asu_destination(self):
        return self.rng.choice(['Rehab', 'ESD', 'Other'], p=self.post_asu_probabilities)

    def sample_length_of_stay(self, destination=None):
        if self.name == 'Stroke':
            mean, std = self.los_params[destination]
        else:
            mean, std = self.los_params

        # Convert lognormal parameters
        normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
        normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))

        return self.rng.lognormal(normal_mean, normal_std)

class ASUModel:
    def __init__(self, env, patient_params=None, run_length=5 * 365):
        self.env = env
        self.run_length = run_length
        self.total_arrivals = 0
        
        default_params = {
            'Stroke': {
                'interarrival_time': 1.2,
                'post_asu_probabilities': [0.24, 0.13, 0.63],
                'los_params': {'Rehab': (7.4, 8.6), 'ESD': (4.6, 4.8), 'Other': (7.0, 8.7)}
            },
            'TIA': {
                'interarrival_time': 9.3,
                'post_asu_probabilities': [0.01, 0.01, 0.98],
                'los_params': (1.8, 5.0)
            },
            'Complex Neurological': {
                'interarrival_time': 3.6,
                'post_asu_probabilities': [0.11, 0.05, 0.84],
                'los_params': (4.0, 5.0)
            },
            'Other': {
                'interarrival_time': 3.2,
                'post_asu_probabilities': [0.05, 0.10, 0.85],
                'los_params': (3.8, 5.2)
            }
        }
        
        if patient_params is None:
            patient_params = default_params
        else:
            # Merge provided params with defaults
            for patient_type, params in default_params.items():
                if patient_type in patient_params:
                    params.update(patient_params[patient_type])
                patient_params[patient_type] = params

        self.patient_types = {
            name: PatientType(name, 
                              params['interarrival_time'], 
                              params['post_asu_probabilities'], 
                              params['los_params'])
            for name, params in patient_params.items()
        }

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            post_asu_destination = patient_type.sample_post_asu_destination()

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Post-ASU destination: {post_asu_destination}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")

            # Call acute treatment process
            self.env.process(self.acute_treatment(patient_type, patient_id, post_asu_destination))

    def acute_treatment(self, patient_type, patient_id, post_asu_destination):
        if patient_type.name == 'Stroke':
            yield from self.stroke_acute_treatment(patient_type, patient_id, post_asu_destination)
        elif patient_type.name == 'TIA':
            yield from self.tia_acute_treatment(patient_type, patient_id)
        elif patient_type.name == 'Complex Neurological':
            yield from self.complex_neurological_acute_treatment(patient_type, patient_id)
        else:  # Other
            yield from self.other_acute_treatment(patient_type, patient_id)

    def stroke_acute_treatment(self, patient_type, patient_id, post_asu_destination):
        los = patient_type.sample_length_of_stay(post_asu_destination)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) finished acute treatment")

    def tia_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) finished acute treatment")

    def complex_neurological_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) finished acute treatment")

    def other_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) finished acute treatment")

def main(patient_params=None, run_length=5*365):
    env = simpy.Environment()
    model = ASUModel(env, patient_params, run_length)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.03: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: ESD
  Next Stroke arrival in 0.03 days
Time 0.03: Patient 0 (Stroke) starting acute treatment
  Length of stay: 5.75 days
Time 1.77: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: Other
  Next Stroke arrival in 1.74 days
Time 1.77: Patient 1 (Stroke) starting acute treatment
  Length of stay: 2.95 days
Time 1.87: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Post-ASU destination: ESD
  Next Stroke arrival in 0.10 days
Time 1.87: Patient 2 (Stroke) starting acute treatment
  Length of stay: 3.79 days
Time 2.11: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Post-ASU destination: Other
  Next Stroke arrival in 0.24 days
Time 2.11: Patient 3 (Stroke) starting acute treatment
  Length of stay: 6.51 days
Time 3.99: Patient 4 (Other) arrived
  Total arrivals: 5
  Other arrivals: 1
  Post-ASU destinat

## Testing lognormal snippets

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [6]:
def test_lognormal_moments(mean, std):
   
    # Lognormal class from sim-tools.
    expected_moments = Lognormal(mean, std)
    print(expected_moments.mu, expected_moments.sigma)

    # Convert lognormal parameters
    normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
    normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))
    print(normal_mean, normal_std)

    return (normal_mean, normal_std) == (expected_moments.mu, expected_moments.sigma)


In [7]:
test_lognormal_moments(128.79, 267.51)

4.022977459259615 1.2924439729790718
4.022977459259615 1.2924439729790718


True

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [8]:
M = 10_000_000

In [9]:
def test_1(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': 1.2},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [10]:
test_1(M)

Time 0.43: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.43 days
Time 0.43: Patient 0 (Stroke) starting acute treatment
  Length of stay: 2.08 days
Time 0.54: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: Other
  Next Stroke arrival in 0.11 days
Time 0.54: Patient 1 (Stroke) starting acute treatment
  Length of stay: 12.02 days
Time 2.50: Patient 0 (Stroke) finished acute treatment
Time 3.18: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Post-ASU destination: Other
  Next Stroke arrival in 2.63 days
Time 3.18: Patient 2 (Stroke) starting acute treatment
  Length of stay: 3.06 days
Time 4.31: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Post-ASU destination: Other
  Next Stroke arrival in 1.14 days
Time 4.31: Patient 3 (Stroke) starting acute treatment
  Length of stay: 5.39 days
Time 5.16: Patient 4 (Stroke) arrive

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [15]:
def test_2(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': large_number},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [16]:
test_2(M)


Simulation completed.
Total simulation time: 730.00 days
Total patient arrivals: 0
Total Stroke arrivals: 0
Total TIA arrivals: 0
Total Complex Neurological arrivals: 0
Total Other arrivals: 0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [17]:
TEN_DAYS = 10

In [18]:
def test_3(run_length):
    # Create the simulation environment
    if __name__ == "__main__":
        main(run_length=run_length)  # Run for 2 years with custom parameters


In [19]:
test_3(TEN_DAYS)

Time 1.07: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 1.07 days
Time 1.07: Patient 0 (Stroke) starting acute treatment
  Length of stay: 3.98 days
Time 1.70: Patient 1 (TIA) arrived
  Total arrivals: 2
  TIA arrivals: 1
  Post-ASU destination: Other
  Next TIA arrival in 1.70 days
Time 1.70: Patient 1 (TIA) starting acute treatment
  Length of stay: 0.40 days
Time 1.70: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 2
  Post-ASU destination: Other
  Next Stroke arrival in 0.64 days
Time 1.70: Patient 2 (Stroke) starting acute treatment
  Length of stay: 4.44 days
Time 1.72: Patient 3 (Complex Neurological) arrived
  Total arrivals: 4
  Complex Neurological arrivals: 1
  Post-ASU destination: Other
  Next Complex Neurological arrival in 1.72 days
Time 1.72: Patient 3 (Complex Neurological) starting acute treatment
  Length of stay: 0.92 days
Time 2.10: Patient 1 (TIA) finished acute treatm